In [108]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [109]:
torch.manual_seed(42)

In [110]:
from Catdog_CNN_checkpoint import LeNet
model=LeNet()
model.load_state_dict(torch.load('model.pth'))
for param in model.parameters():
    param.requires_grad = True
print(model)

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [111]:
train_datadir = r'data\train'
test_datadir  = r'data\test'

train_transforms = transforms.Compose([
    transforms.Resize([224, 224]),  # 将输入图片resize成统一尺寸
    transforms.ToTensor(),         
])

test_transforms = transforms.Compose([
    transforms.Resize([224, 224]),  
    transforms.ToTensor(),          
])

train_data = datasets.ImageFolder(train_datadir,transform=train_transforms)

test_data  = datasets.ImageFolder(test_datadir,transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=4)
test_loader  = torch.utils.data.DataLoader(test_data,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=4)


In [112]:
device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.11)


In [113]:
num_epochs = 20

# 进行迁移学习训练过程
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 计算预测结果与真实标签相符的数量
        _, predicted_labels = torch.max(outputs.data, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / (len(train_loader) * train_loader.batch_size)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}, Accuracy: {accuracy}")


Epoch 1/20, Loss: 0.7213496005535126, Accuracy: 0.498
Epoch 2/20, Loss: 0.68920117020607, Accuracy: 0.5354
Epoch 3/20, Loss: 0.6746345734596253, Accuracy: 0.5882
Epoch 4/20, Loss: 0.6600509119033814, Accuracy: 0.614
Epoch 5/20, Loss: 0.6650229179859162, Accuracy: 0.5826
Epoch 6/20, Loss: 0.6608418476581573, Accuracy: 0.6028
Epoch 7/20, Loss: 0.634433982372284, Accuracy: 0.6446
Epoch 8/20, Loss: 0.6270496928691864, Accuracy: 0.6378
Epoch 9/20, Loss: 0.5977966761589051, Accuracy: 0.676
Epoch 10/20, Loss: 0.5352217084169388, Accuracy: 0.725
Epoch 11/20, Loss: 0.4780133980512619, Accuracy: 0.759
Epoch 12/20, Loss: 0.3988454842567444, Accuracy: 0.8094
Epoch 13/20, Loss: 0.35795292884111407, Accuracy: 0.8354
Epoch 14/20, Loss: 0.25635646656155586, Accuracy: 0.8874
Epoch 15/20, Loss: 0.16097065106034278, Accuracy: 0.9326
Epoch 16/20, Loss: 0.17535769443958998, Accuracy: 0.9268
Epoch 17/20, Loss: 0.15666129875928164, Accuracy: 0.9396
Epoch 18/20, Loss: 0.05164331179112196, Accuracy: 0.9742
Epo